In [69]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from datetime import datetime
import pickle
import random

rng = np.random.default_rng(342834)

In [70]:
#labels helpers and processing
def pivot_df(df, id_col, ignore_cols=None):
    if not ignore_cols:
        ignore_cols = []
    date_cols = [x for x in df.columns if x not in [id_col] + ignore_cols]
    dfs = []
    for day in date_cols:
        day_df = df[[id_col, day]].rename({day: 'snowpack'}, axis=1)
        day_df['date'] = day
        dfs.append(day_df)
    return pd.concat(dfs)

def daynum_gen(date_time):
    '''converts date time objects to filename'''
    date_time = datetime.fromisoformat(date_time)
    doy = date_time.timetuple().tm_yday
    year = date_time.year
    return str(year) + '{:03d}'.format(doy)

# Get ordered elevation training data
def add_elevation(order, modis):
    order = pd.DataFrame({'modis_idx': order, 'order': [x for x in range(len(order))]})
    order['station_id'] = order['modis_idx'].apply(lambda x: '-'.join(x.split('-')[:-1]))
    order = order.merge(elev_order).sort_values('order')
    ordered_elev = elevation[order['DEM_order'].to_list(), :, :]
    dim = ordered_elev.shape

    return np.concatenate([modis, ordered_elev.reshape(dim[0], 1, dim[1], dim[2])], axis=1)

def y_merger(x, y):
    '''reattach y labels'''
    y = y.rename(columns={"Unnamed: 0":"cell_id"})
    y = pivot_df(y, 'cell_id').dropna()
    y['date']=y['date'].map(daynum_gen)

    y['idx'] = y['cell_id'] + "-" + y['date']
    y = y.set_index('idx')

    x['idx'] = x['cell_id'] +\
         "-" +x['date_long'].astype(str)
    x = x.set_index('idx')

    return x.join(y['snowpack'])

In [77]:
y_path = "C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/"

train_feat = pd.read_csv(y_path + "ground_measures_train_features.csv")
test =pd.read_csv(y_path + "ground_measures_test_features.csv")
submission = pd.read_csv(y_path + "submission_format.csv")
train_y = pd.read_csv(y_path + "train_labels.csv") 
metadata = pd.read_csv(y_path + "ground_measures_metadata.csv")

In [91]:
sent_path = "C:/Users/Matt/Dropbox/SnowComp/SentinelHelper/"

sentinel_trainfeat = pd.read_csv(sent_path + "sent_pp_trainfeat.csv")
sentinel_testfeat = pd.read_csv(sent_path + "sent_pp_testfeat.csv")
sentinel_ylabs = pd.read_csv(sent_path + "sent_pp_ylab.csv")

### Merge back in y labels, mask NAs

In [80]:
sentinel_trainfeat = y_merger(sentinel_trainfeat, train_feat)
sentinel_testfeat = y_merger(sentinel_testfeat, test)
sentinel_ylabs = y_merger(sentinel_ylabs, train_y)

## Define Training and Testing Sets

In [ ]:
#@title Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [90]:
np.array(sentinel_trainfeat['first'])

'[[-14.68666182 -11.28283651 -10.94262238 ...  -8.63001602  -8.1752884\n   -7.50209077]\n [-14.06172072 -13.67520101 -10.7870076  ...  -6.45086553  -6.06331967\n   -9.75055821]\n [-13.27421134 -11.89650905 -11.98049499 ...  -8.39559617  -6.59923726\n   -9.82995084]\n ...\n [ -7.95123448  -7.3295684   -8.78284667 ... -10.66935572 -11.50698151\n  -11.69684473]\n [-10.30231037  -9.3643654   -8.99757735 ... -10.72397404 -10.92480873\n  -11.77672265]\n [-11.10299803  -9.67240732 -10.89589824 ... -11.86795424 -11.44884082\n  -12.82893485]]'

## Define CNN